In [19]:
import pandas as pd
import numpy as np
import sys, os , time
from tabulate import tabulate

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(folder+file_name)
    print(tabulate(table, headers='keys', tablefmt='rst')) #latex

In [20]:
!ls

CIFAR_tol3     old_executions	   run_CIFAR_v2.sh  simple
CIFAR_tol3_v2  Plot_Results.ipynb  run_LabelMe.sh
LabelMe        run_CIFAR.sh	   run_simple.sh


In [10]:
!ls LabelMe

LabelMe_DS_test.csv		  LabelMe_OursGlobal_trainAnn.csv
LabelMe_DS_train.csv		  LabelMe_OursGlobal_train.csv
LabelMe_hardMV_test.csv		  LabelMe_Raykar_test.csv
LabelMe_hardMV_train.csv	  LabelMe_Raykar_trainAnn.csv
LabelMe_OursGlobal2_testAux.csv   LabelMe_Raykar_train.csv
LabelMe_OursGlobal2_test.csv	  LabelMe_softMV_test.csv
LabelMe_OursGlobal2_trainAnn.csv  LabelMe_softMV_train.csv
LabelMe_OursGlobal2_train.csv	  LabelMe_UpperBound_test.csv
LabelMe_OursGlobal_testAux.csv	  LabelMe_UpperBound_train.csv
LabelMe_OursGlobal_test.csv	  result.txt


In [33]:
folder = "LabelMe/"
files = sorted(os.listdir(folder))

In [15]:
for s in range(1,10):
    print("\n****************************** Scenario "+str(s)+" ******************************")
    scenario_files = find_match("s"+str(s)+".csv",files)
    
    #all from softMV
    aux = find_match("softMV",scenario_files)
    for value in aux:
        plot_files(value)

    #all from hardMV
    aux = find_match("hardMV",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("DS",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("Raykar",scenario_files)
    for value in aux:
        plot_files(value)
        
    aux = find_match("Ours1",scenario_files)
    for value in aux:
        plot_files(value)        
    aux = find_match("Ours2",scenario_files)
    for value in aux:
        plot_files(value)
    aux = find_match("Ours3",scenario_files)
    for value in aux:
        plot_files(value)


****************************** Scenario 1 ******************************

****************************** Scenario 2 ******************************

****************************** Scenario 3 ******************************

****************************** Scenario 4 ******************************

****************************** Scenario 5 ******************************

****************************** Scenario 6 ******************************

****************************** Scenario 7 ******************************

****************************** Scenario 8 ******************************

****************************** Scenario 9 ******************************


In [69]:
methods_list = ["Ann(tr)","softMV","hardMV", "DS", "Raykar", "OursGlobal", ""]
methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","",]

def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(folder+archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return np.mean(result_list)
    except:
        return "-"

def compare_table(tipo,repre="normal"):
    t = pd.DataFrame()
    t["Methods"] = methods_list
    for s in range(1,10):
        t["Scenario "+str(s)] = ["-" for _ in range(len(methods_list))]
        if tipo == 'acc':
            t.loc[0,"Scenario "+str(s)] =  calculate_aux("result_scenario"+str(s)+".txt",what_to_do="mv")

        scenario_files = find_match("s"+str(s)+".csv",files)

        for k, method in enumerate(methods_list[1:]):
            aux = find_match(method,scenario_files)
            if tipo == "acc": #como normal?
                test_aux = find_match("_test_",aux)
                if len(test_aux) != 0:
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+test_aux[0])["Accuracy"][0], 4)
                    
            elif tipo == 'kl' or tipo=='js':
                kl_aux = find_match("_train_",aux)
                if len(kl_aux) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    if tipo == 'kl':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"KL"][0] ,4)
                        except:
                            pass
                    elif tipo=='js':
                        try:
                            t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+kl_aux[0])[to_extract+"JS"][0] ,4)
                        except:
                            pass
            elif tipo == 'accann':
                train_ann = find_match("_trainAnn_",aux)
                if len(train_ann) != 0:
                    if repre=="wmean":
                        to_extract = "Wmean "
                    elif repre=='mean':
                        to_extract="Mean "
                    t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_ann[0])[to_extract+"ACC imiting Annot"][0] ,4)
            elif tipo == 'rmse':
                train_aux = find_match("_train_",aux)
                if len(train_aux) != 0:
                    try:
                        t.loc[k+1,"Scenario "+str(s)] = round(pd.read_csv(folder+train_aux[0])["Average RMSE"][0] ,4)
                    except:
                        pass
            elif tipo == "epochs":
                t.loc[k+1,"Scenario "+str(s)]=calculate_aux("result_scenario"+str(s)+".txt",what_to_do="epochs",model=methods_list_aux[k])
    return t
t_acc = compare_table("acc","normal")
t_kl_mean = compare_table("kl","mean")
t_js_mean = compare_table("js","mean")
t_kl_wmean = compare_table("kl","wmean")
t_js_wmean = compare_table("js","wmean")
t_acc

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [30]:
t_js_mean #normal

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1441,0.1908,0.119,0.1475,0.1613,0.2035,0.1252,-,-
2,hardMV,0.157,0.1974,0.1235,0.1623,0.145,0.1989,0.1361,-,-
3,DS,0.0029,0.0036,0.0046,0.1155,0.531,0.1394,0.1375,-,-
4,Raykar,0.0065,0.0071,0.0053,0.0893,0.1387,0.1051,0.0739,-,-
5,OursGlobal,0.1078,0.1463,0.148,0.1176,0.1269,0.1453,0.0982,-,-
6,,0.0029,0.0036,0.0046,0.1155,0.531,0.1394,0.1375,-,-


In [8]:
t_accAnn_mean = compare_table("accann","mean")
t_accAnn_wmean = compare_table("accann","wmean")
t_rmseAnn = compare_table("rmse","normal")
t_accAnn_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,0.6662,0.7165,0.6292,0.7291,0.7996,0.7483,0.6817,-,-
5,OursGlobal,0.6247,0.6199,0.6001,0.6979,0.7516,0.6993,0.6629,-,-
6,Ours2,-,-,-,-,-,-,-,-,-
7,Ours3,-,-,-,-,-,-,-,-,-


In [35]:
t_epochs = compare_table("epochs")
t_epochs

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,-,-,-,-,-,-,-,-,-
2,hardMV,-,-,-,-,-,-,-,-,-
3,DS,-,-,-,-,-,-,-,-,-
4,Raykar,-,-,-,-,-,-,-,-,-
5,OursGlobal,-,-,-,-,-,-,-,-,-
6,,-,-,-,-,-,-,-,-,-


In [25]:
print(tabulate(t_acc,headers='keys',tablefmt='latex'))

\begin{tabular}{rlrrrrrrrll}
\hline
    & Methods    &   Scenario 1 &   Scenario 2 &   Scenario 3 &   Scenario 4 &   Scenario 5 &   Scenario 6 &   Scenario 7 & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    &     0.777371 &      0.61335 &      0.78022 &     0.771673 &     0.807082 &     0.622711 &     0.782255 & -            & -            \\
  1 & softMV     &     0.6644   &      0.6081  &      0.6572  &     0.6667   &     0.6764   &     0.5914   &     0.665    & -            & -            \\
  2 & hardMV     &     0.7248   &      0.5947  &      0.7527  &     0.7345   &     0.7297   &     0.5784   &     0.7913   & -            & -            \\
  3 & DS         &     0.9482   &      0.9429  &      0.9464  &     0.9344   &     0.9417   &     0.8413   &     0.9254   & -            & -            \\
  4 & Raykar     &     0.9415   &      0.9418  &      0.9417  &     0.9372   &     0.9308   &     0.8037   &     0.9179   & -            & -            \\
  5 & OursGlobal &     0.90

In [27]:
t_js_mean

,Methods,Scenario 1,Scenario 2,Scenario 3,Scenario 4,Scenario 5,Scenario 6,Scenario 7,Scenario 8,Scenario 9
0,Ann(tr),-,-,-,-,-,-,-,-,-
1,softMV,0.1748,0.1881,0.1131,0.1812,0.1801,0.1937,0.1296,-,-
2,hardMV,0.1835,0.1966,0.1502,0.1537,0.1773,0.1981,0.1482,-,-
3,DS,0.0029,0.0036,0.0046,0.1155,0.531,0.1394,0.1375,-,-
4,Raykar,0.0066,0.0075,0.0055,0.0905,0.1385,0.1055,0.0738,-,-
5,OursGlobal,0.1511,0.1574,0.1466,0.1346,0.1181,0.1377,0.1271,-,-
6,Ours2,-,-,-,-,-,-,-,-,-
7,Ours3,-,-,-,-,-,-,-,-,-


In [31]:
print(tabulate(t_js_mean,headers='keys',tablefmt='latex'))

\begin{tabular}{rllllllllll}
\hline
    & Methods    & Scenario 1   & Scenario 2   & Scenario 3   & Scenario 4   & Scenario 5   & Scenario 6   & Scenario 7   & Scenario 8   & Scenario 9   \\
\hline
  0 & Ann(tr)    & -            & -            & -            & -            & -            & -            & -            & -            & -            \\
  1 & softMV     & 0.1441       & 0.1908       & 0.119        & 0.1475       & 0.1613       & 0.2035       & 0.1252       & -            & -            \\
  2 & hardMV     & 0.157        & 0.1974       & 0.1235       & 0.1623       & 0.145        & 0.1989       & 0.1361       & -            & -            \\
  3 & DS         & 0.0029       & 0.0036       & 0.0046       & 0.1155       & 0.531        & 0.1394       & 0.1375       & -            & -            \\
  4 & Raykar     & 0.0065       & 0.0071       & 0.0053       & 0.0893       & 0.1387       & 0.1051       & 0.0739       & -            & -            \\
  5 & OursGlobal & 0.1078  

In [72]:

#all from softMV
aux = find_match("softMV",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="soft-MV"))

#all from hardMV
aux = find_match("hardMV",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="hard-MV"))


aux = find_match("DS",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="D&S"))

aux = find_match("Raykar",files)
for value in aux:
    plot_files(value)
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="Raykar"))

aux = find_match("OursGlobal_",files)
for value in aux:
    plot_files(value)         
print("Epochs:",calculate_aux("result.txt",what_to_do="epochs",model="Ours Global"))



File to show:  LabelMe_softMV_test.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global          0.817508      0.817508
====  ============  ==========  ============

File to show:  LabelMe_softMV_train.csv
====  ============  ==========  ============  =========  =========  ==========  ==========
  ..  Unnamed: 0      Accuracy    F1 (micro)    Mean KL    Mean JS    Wmean KL    Wmean JS
====  ============  ==========  ============  =========  =========  ==========  ==========
   0  Global            0.8328        0.8328    2.27289   0.409569     1.12616    0.342534
====  ============  ==========  ============  =========  =========  ==========  ==========
Epochs: 9.13333333333

File to show:  LabelMe_hardMV_test.csv
====  ============  ==========  ============
  ..  Unnamed: 0      Accuracy    F1 (micro)
====  ============  ==========  ============
   0  Global          0.800589      0.8005